In [ ]:
!pip install --upgrade mxnet
!pip install autogluon

In [ ]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
import pandas as pd

In [ ]:
# Seed for Reproducibility
SEED = 21
# Validation data size in percent
VAL_SIZE = 0.2

In [ ]:
def create_train_val_dataset(dataset, target_column):
    feature_names = dataset.columns.difference([target_column])
    df_train, df_val = train_test_split(dataset, test_size=VAL_SIZE, random_state=SEED)
    return df_train, df_val

In [ ]:

hyperparameters = {
    'GBM': {},
    'CAT': {},
    'RF': {},
    'XT': {},
    'KNN': {},
}
    
def run_autogluon_experiment(dataset,
                             target_column,
                             problem_type,
                             eval_metric,
                             time_limit=600
                            ):
    predictor = TabularPredictor(
        label = target_column,
        problem_type = problem_type,
        eval_metric = eval_metric,
        verbosity=2
    )
    predictor.fit(
        train_data = dataset,
        time_limit = time_limit,
        presets='high_quality_fast_inference_only_refit',
        excluded_model_types=['KNN', 'custom'],
        # hyperparameters = hyperparameters
    )
    return predictor

In [ ]:
# Constants

# Outcome variable
TARGET = 'pressure'

PROBLEM_TYPE='regression'

EVAL_METRIC='mean_absolute_error'

In [ ]:
df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df.head()

In [ ]:
df_train, df_val = create_train_val_dataset(df, TARGET)
df_train.shape, df_val.shape

In [ ]:
predictor = run_autogluon_experiment(dataset=df_train,
                                     target_column=TARGET,
                                     problem_type=PROBLEM_TYPE,
                                     eval_metric=EVAL_METRIC,
                                     )

In [ ]:
_ = predictor.leaderboard()

In [ ]:
val_performance = predictor.evaluate(df_val)
print('MAE:', val_performance[EVAL_METRIC])

In [ ]:
df_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

y_pred = predictor.predict(df_test)
y_pred_submission = pd.concat([df_test[['id']], pd.DataFrame(y_pred, columns=['pressure'])], axis=1)
y_pred_submission.head()

In [ ]:
y_pred_submission.to_csv('submission.csv', index=False)